# Benchmark Models

This notebook implements simple benchmark models to compare against LSTM performance:

In [ ]:
#| default_exp benchmark_model

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.basics import patch
from pathlib import Path
import hvplot.pandas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler

In [ ]:
#| export
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

We will first open the data

In [ ]:
DATA_PATH = Path("../testing_data")

df = pd.read_csv(
    DATA_PATH/'hydro_example.csv', 
    usecols=['time', 'smoothed_rain', 'Q_mgb', 'Q_obs'], 
    index_col='time',
    converters={"time": pd.to_datetime}
    )

We can now take a quick look to the data to see the trends and the expected improvement

In [ ]:

def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

normalize(df).plot()#.hvplot.line()

## Data preprocessing

Select feature and target columns

In [ ]:
x_col, y_col = ['smoothed_rain','Q_mgb'], ['Q_obs']

Split data

In [ ]:
train_mask = df.index < '2019-01-01'
train = df[train_mask]
valid = df[~train_mask]

Scale data

In [ ]:
features_scaler = RobustScaler()
train[x_col] = features_scaler.fit_transform(train[x_col])
valid[x_col] = features_scaler.transform(valid[x_col])

## Feature generator
> Feature generator that uses a sliding window of n previous timesteps and polynomial features to predict the next value, enabling learning of temporal patterns and non-linear relationships

In [ ]:
#| export
class FeatureAndTargetGenerator:
    """
    Transforms time series data into feature matrices suitable for machine learning models.
    Creates lagged features using a sliding window and optionally generates polynomial features
    to capture non-linear relationships between variables.
    """
    def __init__(self, context_len: int = 10, target_len: int = 10, poly_degree: int = 1):
        self.context_len = context_len
        self.target_len = target_len
        self.poly_features = PolynomialFeatures(degree=poly_degree)
        
    def generate(self, df: pd.DataFrame, x_col: list[str], y_col: list[str]) -> tuple[pd.DataFrame, pd.DataFrame]:
        X, y = df[x_col], df[y_col]
        if 1 < self.poly_features.degree:
            X = pd.DataFrame(self.poly_features.fit_transform(X), index=X.index)
        X, y = self.generate_sliding_window_features(X, y)
        return X, y


    def generate_sliding_window_features(self, X: pd.DataFrame, y: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        Creates a feature matrix by combining multiple input variables and their lagged values.
        For each time step t, takes values from t-window to t for each input variable
        and combines them into a single feature vector. The target value is taken at time t.
        This allows the model to learn patterns across multiple timesteps.
        """
        features = []
        targets = []
        
        for i in range(len(X) - self.context_len - self.target_len):
            row_features = X.iloc[i:i + self.context_len]
            features.append(row_features.values.reshape(-1))
            row_targets = y.iloc[i + self.context_len: i + self.context_len + self.target_len]
            targets.append(row_targets.values.reshape(-1))

        features = pd.DataFrame(
            index=X.index[self.context_len - 1:len(X) - self.target_len - 1],
              data=features)
        targets = pd.DataFrame(
            index=y.index[self.context_len - 1:len(X) - self.target_len - 1],
            data=targets,
            columns=[f"t+{i+1}" for i in range(0, self.target_len)])
        return features, targets
    


For example we can generate train and validation sets as follows

In [ ]:
generator = FeatureAndTargetGenerator(context_len=1, target_len=2, poly_degree=2)
train_x, train_y = generator.generate(train, x_col, y_col)
valid_x, valid_y = generator.generate(valid, x_col, y_col)
train_x.head(3), train_y.head(3)

### Benchmark model
We chose a linear regression model as our benchmark because:
1. When combined with our feature generator, it provides a simple yet effective baseline
2. The polynomial features allow it to capture non-linear relationships in the data
3. The sliding window enables learning of temporal patterns across multiple timesteps
4. Linear regression models are highly interpretable and computationally efficient

In [ ]:
#| export
class SimpleRegressionModel:
    def __init__(self):
        self.model = LinearRegression()
    
    def fit(self, X, y=None):
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        return self.model.predict(X)
    
    def predict_as_dataframe(self, X, degree, context_len):
        pred = self.predict(X)
        pred = pd.DataFrame(pred, index=X.index, columns=[f"t+{i+1}" for i in range(0, pred.shape[1])])
        pred.columns.name = "forecast_horizon"
        pred.index.name = "run_time"
        pred['degree'] = degree
        pred['context_len'] = context_len
        pred = pred.set_index(['degree', 'context_len'], append=True)
        pred = pred.reorder_levels(["degree", "context_len", "run_time"]).sort_index()
        return pred

We can define and train a simple model as follows

In [ ]:
model = SimpleRegressionModel()
model.fit(train_x, train_y)

And we can get the raw prediction

In [ ]:
model.predict(valid_x)[:3]

Or the prediction with indexes as follows

In [ ]:
pred = model.predict_as_dataframe(valid_x, degree=2, context_len=1)
pred.head(3)

## Time index handling

This section implements utilities for handling time indexes in forecasting data. We provide functions to: Convert between two data formats:
- Forecast horizons as columns (e.g., t+1, t+2 columns)
- Forecast horizons and times as row indices
The first format is convenient for saving data, while the second is better suited for scoring and plotting since it explicitly tracks the actual forecast times.

In [ ]:
#| export
class ForecastTimeHandler:
    """
    A utility class for handling forecast time transformations.

    This class provides functionality to manipulate forecast time between different formats,
    specifically handling the conversion between columnar forecast horizons and stacked time series formats.
    It manages forecast horizons (e.g., 't+1', 't+2') and their corresponding timestamps.
    """
    def __init__(
            self,
            stack_col_name: str = "pred" # Name of the column when columns are stacked
            ):
        self.stack_col_name = stack_col_name

    def stack(self, df: pd.DataFrame) -> pd.DataFrame:
        """Stack the forecast horizon as index and add forecast time as index"""
        df = df.copy()
        if df.columns.name is None:
            df.columns.name = "forecast_horizon"
        df = self.transpose_forecast_horizon_as_index(df)
        df = self.add_forecast_time_as_index(df)
        return df
    
    def transpose_forecast_horizon_as_index(self, df: pd.DataFrame) -> pd.DataFrame:
        df=df.stack()
        df = df.to_frame(self.stack_col_name)
        return df

    def add_forecast_time_as_index(self, df: pd.DataFrame) -> pd.DataFrame:
        def get_daily_timedeltas(forecast_horizons):
            """Extract timedelta day values from forecast horizons starting with 't+'"""
            return [pd.Timedelta(days=int(fh.replace("t+", ""))) for fh in forecast_horizons if fh.startswith('t+')]
        df = df.copy()
        forecast_horizon = df.index.get_level_values("forecast_horizon")
        timedeltas = get_daily_timedeltas(forecast_horizon)
        df["forecast_time"] = df.index.get_level_values("run_time") + pd.Index(timedeltas)
        df.set_index("forecast_time", inplace=True, append=True)
        return df
    
    def unstack(self, df: pd.DataFrame) -> pd.DataFrame:
        """Convert stacked forecast horizon index back to horizon-as-columns format"""
        return df.reset_index("forecast_time", drop=True)[self.stack_col_name].unstack("forecast_horizon")
    
    def align(self, pred, obs):
        """Align the predictions and observations by forecast time"""
        obs, pred = obs.copy(), pred.copy()
        obs.index.name = "forecast_time"
        pred, obs = pred.align(obs, join="outer", axis=0)
        return pred, obs
    
    def join(self, pred, obs):
        """Join the predictions and observations by forecast time"""
        obs, pred = obs.copy(), pred.copy()
        obs.index.name = "forecast_time"
        return pred.join(obs, on="forecast_time")

    def align_as_xarray(self, pred, obs):
        """Align the predictions and observations by forecast horizon and return as xarray"""
        if 1 != len(obs.columns):
            raise ValueError("Observations must have only one column")
        obs_col = obs.columns[0]
        pred_col = self.stack_col_name

        obs, pred = obs.copy(), pred.copy()
        pred = self.stack(pred)
        obs.index.name = "forecast_time"
        pred, obs = self.align(pred, obs)

        self.stack_col_name = obs_col
        obs = self.unstack(obs)
        obs = obs.to_xarray().to_array("forecast_horizon", name=obs_col)
        self.stack_col_name = pred_col
        pred = self.unstack(pred)
        pred = pred.to_xarray().to_array("forecast_horizon", name=pred_col)
        
        return pred, obs
    
    def align_and_join_as_xarray(self, pred, obs):
        """Align the predictions and observations by forecast horizon and join them as xarray"""
        pred, obs = self.align_as_xarray(pred, obs)
        return xr.merge([pred, obs])


The previous class can be used to add forecast time as index and to revert this operation. First we create an instance of the class giving the name of the column when forecast horizon will be set as index:

In [ ]:
forecast_time_handler = ForecastTimeHandler(stack_col_name="pred")

Now we can use the instance to stack the forecast horizon as index:

In [ ]:
stacked_pred = forecast_time_handler.stack(pred)
stacked_pred.head(5)


Finally we can rever the operation as follows

In [ ]:
forecast_time_handler.align_and_join_as_xarray(pred, observations)

## Scoring

Since we have generated multiple predictions with different parameters, we will select only the best performing models according to each metric

In [ ]:
#| export
class BenchmarkScores:
    """
    A class to compute and compare different error metrics between predictions and observations.
    
    This class provides a flexible way to compute multiple error metrics between predicted and observed values.
    New metrics can be easily added by implementing them as methods.
    The new metrics can then be used by passing their method names as strings to compute_scores().
    
    Example:

    ```python
    class CustomBenchmarkScores(BenchmarkScores):
        def mse(self, pred, obs):
            error = ((pred - obs)**2).mean(dim="time") 
            return error
        
    # Can then be used as:
    scores = compute_scores(pred, obs, metrics=["mae", "rmse", "mse"])
    ```
    """
    def __init__(self): pass


    def compute_scores(self, predictions: pd.DataFrame, observations: pd.DataFrame, metrics: list[str]) -> pd.DataFrame:
        """Compute the scores for the predictions and observations."""
        forecast_time_handler = ForecastTimeHandler()
        pred, obs = forecast_time_handler.align_as_xarray(predictions, observations)
        
        scores = []
        for metric in metrics:
            score = getattr(self, metric)(pred, obs)
            scores.append(score.to_dataset(name=metric))
        return xr.merge(scores).to_dataframe()
    
    def mae(self, pred, obs):
        error = abs(pred - obs).mean(dim="run_time")
        return error

    def rmse(self, pred, obs):
        error = np.sqrt(((pred - obs)**2).mean(dim="run_time"))
        return error
    
    def nse(self, pred, obs):
        """
        Calculate Nash-Sutcliffe Efficiency score.
        
        $NSE = 1 - \\frac{\\sum(pred - obs)^2}{\\sum(obs - \\overline{obs})^2}$
        
        NSE ranges from -inf to 1, with 1 being a perfect match
        """
        numerator = ((pred - obs)**2).sum(dim="run_time")
        denominator = ((obs - obs.mean(dim="run_time"))**2).sum(dim="run_time")
        error = 1 - (numerator / denominator)
        return error
    
    def kge(self, pred, obs):
        """Calculate Kling-Gupta Efficiency score.
        
        $KGE = 1 - sqrt((r-1)^2 + (\alpha-1)^2 + (\beta-1)^2)$
        
        where:
        - r = Pearson correlation coefficient
        - $\\alpha = \\frac{std(pred)}{std(obs)}$ 
        - $\\beta = \\frac{mean(pred)}{mean(obs)}$
        
        KGE ranges from -inf to 1, with 1 being a perfect match
        """
        # Calculate components
        r = xr.corr(pred, obs, dim="run_time")
        alpha = pred.std(dim="run_time") / obs.std(dim="run_time")
        beta = pred.mean(dim="run_time") / obs.mean(dim="run_time")
        
        # Calculate KGE
        error = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
        return error

    def find_nbest_scores(
            self,
            df: pd.DataFrame, # Dataframe containing scores as columns
            how: dict[str, str], # Dictionary of how to find the best scores for each variable
            n: int=2 # Number of best scores to find
        ):
        """Given a dataset containing scores as variables,find the n best scores for each score."""
        assert all(how in ["min", "max"] for how in how.values()), "how must be either 'min' or 'max'"
        df = df.reorder_levels(["forecast_horizon"] + [level for level in df.index.names if level != "forecast_horizon"])
        best_scores_coords = []
        for score, how in how.items():
            ascending = how == "min"
            best_scores_forecast_horizon = self._find_best_scores_by_forecast_horizon(df[score], n, ascending)
            best_scores_coords.extend(best_scores_forecast_horizon.values.tolist())
        best_scores_coords = pd.MultiIndex.from_tuples(best_scores_coords, names=df.index.names)
        n_best_scores = df.loc[best_scores_coords]
        n_best_scores = n_best_scores.sort_index().drop_duplicates()
        return n_best_scores

    def _find_best_scores_by_forecast_horizon(self, serie: pd.Series, n, ascending: bool):
        """For each unique forecast horizon in a multi-indexed Series, find indices of the n best values based on ascending/descending sort."""
        sorted = serie.sort_values(ascending=ascending)
        nbest_values = []
        for v in sorted.index.get_level_values("forecast_horizon").unique():
            nbest_values.append(sorted.loc[[v]].head(n))
        nbest_values = pd.concat(nbest_values)
        return nbest_values.index

In [ ]:
benchmark_scores = BenchmarkScores()


In [ ]:
_ = benchmark_scores.compute_scores(predictions, observations, ["mae", "rmse", "nse", "kge"])

In [ ]:
benchmark_scores.find_nbest_scores(_, how={"mae": "min", "rmse": "min", "nse": "max", "kge": "max"}, n=1)

In [ ]:
scores_ds = benchmark_scores.compute_scores(
    predictions,
    observations["Q_obs"],
    ["mae", "rmse", "nse", "kge"])
best_scores = benchmark_scores.find_nbest_scores(
    scores_ds,
    how={"mae": "min", "rmse": "min", "nse": "max", "kge": "max"},
    n=1)

## Model training and comparison

We can now make a simple prediction to compare different window sizes and differen polynomial degrees

In [ ]:
predictions = []
scores = []
for degree in range(1, 2+1):
    for window in range(10, 20+1, 10):
        feature_generator = FeatureGenerator(context_window=window, target_window=4, degree=degree)        
        train_x, train_y = feature_generator.generate(train, x_col, y_col)
        valid_x, valid_y = feature_generator.generate(valid, x_col, y_col)

        model = SimpleRegressionModel()
        model.fit(train_x, train_y)
        
        pred = model.predict_as_dataframe(valid_x, degree, window)
        predictions.append(pred)

predictions = pd.concat(predictions)#.to_xarray()
observations = valid[y_col]
# predictions = pd.concat(predictions).to_xarray()
# observations = valid[y_col].to_xarray().sel(time=slice(predictions.run_time.min(), predictions.run_time.max()))

#### Static plotting

Finally we will plot the error so we can visualize the best results

In [ ]:
#| export
def plot_static_benchmark_scores(
        df: pd.DataFrame, # Dataframe with polynomial degree and window as index and mae and mse as columns
        figsize: tuple=(8, 7), # Figure size in inches (width, height)
        fontsize: int=7, # Font size for annotations
        xlim: tuple=(None, None), # Tuple of (min, max) values for x-axis limits
        ylim: tuple=(None, None), # Tuple of (min, max) values for y-axis limits
        ):
    """Plot MAE vs MSE scores with degree and window annotations for model comparison"""
    fig, ax = plt.subplots(figsize=figsize)
    
    # Get unique time steps - expected format 't+N' where N is 0-10
    time_steps = []
    for col in df.index.get_level_values(0).unique():
        if isinstance(col, str) and col.startswith('t+'):
            time_steps.append(col)
    time_steps = sorted(time_steps, key=lambda x: int(x.split('+')[1]))
    
    # Create a colormap
    colors = plt.cm.viridis(np.linspace(0, 1, len(time_steps)))
    
    # Plot each time step with different color
    for t, color in zip(time_steps, colors):
        mask = df.index.get_level_values(0) == t
        df_t = df[mask]
        scatter = ax.scatter(df_t['mae'], df_t['rmse'], label=t, color=color)
        
        # Add annotations for each point
        for (_, deg, win), (mae, rmse) in zip(df_t.index, df_t[['mae', 'rmse']].values):
            ax.annotate(f'd={deg},w={win}', (mae, rmse), 
                       xytext=(5, 5), textcoords='offset points', 
                       fontsize=fontsize, color=color)

    ax.set(xlabel='MAE', ylabel='RMSE', 
           title='MAE vs RMSE for different degrees, windows and prediction horizons')
    ax.legend(title='Prediction horizon', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    plt.tight_layout()
    plt.show()

In [ ]:
#| hide
#| eval: false
plot_static_benchmark_scores(best_scores)

#### Interactive plotting

In [ ]:
#| export
def plot_interactive_benchmark_scores(
        df: pd.DataFrame, # Dataframe with polynomial degree and window as index and mae and mse as columns
        figsize: tuple=(800, 600), # Figure size in pixels (width, height)
        fontsize: int=12, # Font size for annotations
        xlim: tuple=(None, None), # Tuple of (min, max) values for x-axis limits
        ylim: tuple=(None, None), # Tuple of (min, max) values for y-axis limits
        ):
    """Plot MAE vs MSE scores with degree and window annotations for model comparison using Plotly"""
    import plotly.graph_objects as go
    import plotly.express as px
    
    # Get unique time steps - expected format 't+N' where N is 0-10
    time_steps = []
    for col in df.index.get_level_values(0).unique():
        if isinstance(col, str) and col.startswith('t+'):
            time_steps.append(col)
    time_steps = sorted(time_steps, key=lambda x: int(x.split('+')[1]))
    # Create a colormap
    # Generate more colors by interpolating the Viridis colormap
    n_colors = len(time_steps)
    colors = px.colors.sample_colorscale('Viridis', n_colors)
    # Create figure
    fig = go.Figure()
    
    # Plot each time step with different color
    for t, color in zip(time_steps, colors):
        mask = df.index.get_level_values(0) == t
        df_t = df[mask]
        
        # Add scatter plot
        fig.add_trace(go.Scatter(
            x=df_t['mae'],
            y=df_t['rmse'],
            mode='markers+text',
            name=t,
            marker=dict(color=color, size=10),
            text=[f'd={deg},w={win}' for (_, deg, win) in df_t.index],
            textposition="top center",
            textfont=dict(size=fontsize),
            hovertemplate="MAE: %{x:.3f}<br>RMSE: %{y:.3f}<br>%{text}<extra></extra>"
        ))
    
    # Update layout
    fig.update_layout(
        width=figsize[0],
        height=figsize[1],
        title='MAE vs RMSE for different degrees, windows and prediction horizons',
        xaxis_title='MAE',
        yaxis_title='RMSE',
        legend_title='Prediction horizon',
        hovermode='closest',
        showlegend=True
    )
    
    # Set axis limits if provided
    if xlim[0] is not None or xlim[1] is not None:
        fig.update_xaxes(range=xlim)
    if ylim[0] is not None or ylim[1] is not None:
        fig.update_yaxes(range=ylim)
    
    return fig


In [ ]:
plot_interactive_benchmark_scores(best_scores,)


## Time series verification

Now that we have identified the optimal model parameters, let's verify its performance by comparing the predicted discharge values with both observed values and MGB model predictions. This comparison will be done across the full prediction horizon to assess how well our model maintains its predictive power over time. We'll visualize these comparisons using time series plots that show the observed discharge, our model's predictions, and the MGB model predictions side by side.


In [ ]:
#| export
def plot_prediction_comparison(
        observed: xr.DataArray, # Time series of observed water discharge values
        predicted: xr.Dataset, # Dataset containing predicted discharge values for each forecast horizon (t+i)
        mgb: xr.DataArray, # Time series of MGB model water discharge predictions
        best_model: dict=None,
        scores: xr.Dataset=None # Dictionary containg the t+i as key and the best model fit degree and window as value
        ) -> plt.Figure:
    """Plot comparison between observed, predicted and MGB discharge values for a i-day horizon."""
    n_horizon = len(predicted.data_vars)
    fig, axes = plt.subplots(int(n_horizon/2), 2, figsize=(20, int(n_horizon/2*5)), sharex=True, sharey=True)
    axes = axes.flatten()

    for i in range(0, n_horizon):
        y_obs = observed.to_series()
        y_pred = predicted[f"t+{i+1}"]
        if best_model is not None:
            y_pred = y_pred.sel(**best_model[f"t+{i+1}"])
        y_pred = y_pred.to_series()
        y_mgb = mgb.to_series()


        if scores is not None:
            score = scores.sel(forecast_horizon=f"t+{i+1}")
            if best_model is not None:
                score = score.sel(**best_model[f"t+{i+1}"])
            score = score.to_array().to_series().to_dict()

        axes[i].plot(y_obs, label="Q_obs", color='blue')
        axes[i].plot(y_pred, label="Q_pred", color='red', linestyle='solid')
        axes[i].plot(y_mgb, label="Q_mgb", color='black')

        score_str = "".join(f" {k.upper()}={v:.2f}" for k, v in score.items())
        if best_model is not None:
            model_str = "".join(f" {k.capitalize()}={v}" for k, v in best_model[f"t+{i+1}"].items())
        else:
            model_str = ""
        axes[i].set_title(f"Jour {i+1} {model_str}\n {score_str}")
        
        axes[i].grid()
        axes[i].legend()

    plt.suptitle("Comparaison des valeurs réelles et prédites pour chaque jour de l'horizon de 10 jours", fontsize=14)
    plt.tight_layout()
    return fig

From the previous analysis we can decides which model works the best for each predicted time step. We will use the best model for each time step to plot the prediction comparison. As example:

In [ ]:
best_models={
    "t+1": {"degree": 2, "ctx_window": 20}, 
    "t+2": {"degree": 2, "ctx_window": 20}, 
    "t+3": {"degree": 2, "ctx_window": 10},
    "t+4": {"degree": 2, "ctx_window": 10},
    }
fig = plot_prediction_comparison(
    observed=observations["Q_obs"], 
    predicted=predictions, 
    best_model=best_models, 
    mgb=df[~train_mask]["Q_mgb"].to_xarray(),
    scores=scores_ds
    )
plt.show()

## Save prediction data

We save the best model for later use

In [ ]:
#| hide
#| eval: false
results = predictions.merge(observations)
benchmark_ds = results.sel(degree=2, ctx_window=slice(30,50))
# benchmark_ds.to_netcdf(DATA_PATH/'regression_benchmark.nc')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()